In [ ]:
import requests
import json
import pandas as pd
import altair as alt

In [ ]:
placeholder = "#PLACEHOLDER_PARENT_METADATA_VALUE#"
acronym = "ISI"
scope2 = "f34e8dc2-5a79-45b3-a500-10f867ce3e9f"

# Instituts-Kürzel
#acronym = "IRB"
#scope2 = "fa1960d9-a52f-4a41-af94-2a2a40fa7e67" 

# Jahr von (min) bis (max). Für die Einschränkung auf ein Jahr, bitte den gleichen Wert für date_min und date_max einfügen
#date = "2021"
date_min = "2020" 
date_max = "2020"


url = "https://publica-rest.fraunhofer.de/server/api/discover/search/objects?query=institute:("+acronym+")&configuration=researchoutputs&f.dateIssued=%5B"+date_min+"%20TO%20"+date_max+"%5D,equals"

r = requests.get(url)
data = r.json()
anzahl = data["_embedded"]["searchResult"]["page"]["totalElements"]
anzahl = int(anzahl)
print(anzahl)
pages = data["_embedded"]["searchResult"]["page"]["totalPages"]
pages = int(pages)
#print(pages)
object = data["_embedded"]["searchResult"]["_embedded"]["objects"]

uuid = []
affil = []
authority = []
for j in range(len(object)):
    metadata = data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]
    check = metadata.get("oairecerif.author.affiliation", 0)
    if check != 0:
        test = data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["oairecerif.author.affiliation"]
        for i in range(len(test)):
            uuid.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["uuid"])
            affil.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["oairecerif.author.affiliation"][i]["value"])
            authority.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["oairecerif.author.affiliation"][i]["authority"])

if pages > 1:
    for k in range(pages-1):
        url = data["_embedded"]["searchResult"]["_links"]["next"]["href"]
        r = requests.get(url)
        data = r.json()
        object = data["_embedded"]["searchResult"]["_embedded"]["objects"]
        for j in range(len(object)):
            metadata = data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]
            check = metadata.get("oairecerif.author.affiliation", 0)
            if check != 0:
                test = data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["oairecerif.author.affiliation"]
                for i in range(len(test)):
                    uuid.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["uuid"])
                    affil.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["oairecerif.author.affiliation"][i]["value"])
                    authority.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["oairecerif.author.affiliation"][i]["authority"])
 
        
#Dataframe erstellen
df = pd.DataFrame({'uuid':uuid})
df['name'] = affil
df['authority'] = authority
#print(df)

#Löschen des eigenen Namens
df = df[df.authority != scope2]
#Löschen von Placeholdern
df = df[df.name != placeholder]

#Deduplizieren
df2 = df.drop_duplicates()

#Gewichten: goupby count
#df = df.groupby(['name', 'authority']).size()
df3 = df2.groupby(['name'])['name'].count().reset_index(name='counts')
df3 = df3.reset_index(drop=True)

print(df3)

356
                                                  name  counts
0                                          ATEMIS GmbH       1
1    Aalborg Univ., Section of Sustainable Design a...       1
2                                         Aarhus Univ.       3
3    Anglia Ruskin Univ., Global Sustainability Ins...       1
4    Anhui Univ. of Finance and Economics, School o...       1
..                                                 ...     ...
283                              datenschutz nord GmbH       2
284                     energy & management consulting       1
285                                               ifeu       1
286                                         talanwälte       1
287                                       Öko-Institut       1

[288 rows x 2 columns]


In [ ]:
df3.sort_values(by=['counts'], inplace=True, ascending=False)# Absteigende Sortierung nach Anzahl.
df3.columns = ['Name', 'Anzahl']
print(df3)

                                                  Name  Anzahl
101                          HfWU Nürtingen-Geislingen       6
28                                          DB Netz AG       6
126                                 KIT Karlsruhe, IIP       6
273  Utrecht Univ., Copernicus Institute of Sustain...       4
241                               Univ. Stuttgart, IER       4
..                                                 ...     ...
248                                 Univ. of Bern, CDE       1
249                               Univ. of Bern; Eawag       1
285                                               ifeu       1
284                     energy & management consulting       1
53   ESM Foundation; Univ. Augsburg, Institute for ...       1

[288 rows x 2 columns]


In [ ]:
bars = alt.Chart(df3).mark_bar().encode(
    x='Anzahl:Q',
    y=alt.Y("Name:O", sort='-x')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Anzahl:Q'
)

(bars + text).properties(height=900)

In [ ]:
df4 = df3[df3.Anzahl > 2] # Hier bitte gewünschten Wert eintragen.

bars = alt.Chart(df4).mark_bar().encode(
    x='Anzahl:Q',
    y=alt.Y("Name:O", sort='-x')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Anzahl:Q'
)

(bars + text)
#(bars + text).properties(height=900)

alt.LayerChart(...)